In [5]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
# Bruteforce solution for checking logic
# input variables
lxy = 5.0  # PnC unit cell spacing (mm) 
theta = 20  # PnC filament orintation (degrees)
L = 34.0  # PnC length (mm)
W = 34.0   # PnC width (mm)

# calculate distance between first two waypoints
theta_rad = theta * np.pi/180
wh = 2 * lxy * np.sin(theta_rad)   # converting angles from degrees to radians
wv = 2 * lxy * np.cos(theta_rad)   # converting angles from degrees to radians


# plot waypoints
def plot_waypoints(waypoints):
    # ----------- plot figure --------------- 
    plt.figure(figsize=(6, 6))
    for i in range(len(waypoints) - 1):
        point1 = [waypoints[i][0], waypoints[i+1][0]]
        point2 = [waypoints[i][1], waypoints[i+1][1]]
        plt.plot(point1, point2, '-o', color='black')

    # plt.hlines(0, 0, L, lw = 3.0, color='red')
    # plt.vlines(0, 0, W, lw = 3.0, color='red')
#     plt.hlines(L, 0, L, lw = 3.0, color='red')
    # plt.vlines(W, 0, W, lw = 3.0, color='red')

    x = [i for i in range(35)]
    plt.xticks(x, x)
    x = [i for i in range(35)]
    plt.yticks(x, x)
    plt.show()
    
    
def flip_v(waypoints, h_cond, v_cond, break_idx, idx_h, idx_v):
    print('flip vertical')
    print(f'h_cond: {h_cond}, v_cond: {v_cond}')

    if idx_h * wh > L or idx_v*wv > W:
        return waypoints

    d = (break_idx*wv-W)*np.tan(theta_rad)
    for i in range(5):
        waypoints.append([d+idx_v*wh, W])
        waypoints.append([d+(idx_v+1)*wh, W])
        if (break_idx+1)*wh <= L:
            waypoints.append([(break_idx+1)*wh, 0])
        else:
            break_idx = break_idx + 1
            h_cond = False
            waypoints = flip_h(waypoints, h_cond, v_cond, break_idx, idx_h, idx_v, d)
            break
        if (break_idx+2)*wh <= L:
            waypoints.append([(break_idx+2)*wh, 0])
        else: 
            break_idx = break_idx + 2
            h_cond = False
            waypoints = flip_h(waypoints, h_cond, v_cond, break_idx, idx_h, idx_v)
            break
        break_idx += 2
        idx_v += 2
    return waypoints, d


def flip_h(waypoints, h_cond, v_cond, break_idx, idx_h, idx_v, d):
    print('flip horizontal')
    print(f'h_cond: {h_cond}, v_cond: {v_cond}')

    if idx_h * wh > L or idx_v*wv > W:
        return waypoints

    p = ((break_idx+1)*wh-L)*np.tan(theta_rad)

    for i in range(5):
        if p + idx_h*wv <= L:
            waypoints.append([L, p + idx_h*wv])
        else: break
        if p + (idx_h+1)*wv <= L:
            waypoints.append([L, p + (idx_h+1)*wv])
        else: break
        if d+(idx_v+2)*wh <= W:
            waypoints.append([d+(idx_v+2)*wh, W])
        else: 
            v_cond = False
            waypoints = flip_v(waypoints, h_cond, v_cond, break_idx, idx_h, idx_v)
            break
        if d+(idx_v+3)*wh <= W:
            waypoints.append([d+(idx_v+3)*wh, W])
        else: 
            v_cond = False
            waypoints = flip_v(waypoints, h_cond, v_cond, break_idx, idx_h, idx_v)
            break
        idx_h += 2
        idx_v += 2

    return waypoints

In [6]:
waypoints = []
idx = 0
idx_h = 0
idx_v = 0

h_cond = True
v_cond = True
for i in range(5):
    if idx*wh <= W:
        waypoints.append([idx*wh, 0])
    else: 
        h_cond, term = False, 1
        break_idx = idx
        waypoints = flip_h(waypoints, h_cond, v_cond, break_idx, idx_h, idx_v, d)
        break
    if (idx + 1) * wh <= W:
        waypoints.append([(idx+1)*wh, 0])
    else: 
        h_cond, term = False, 2
        break_idx = idx + 1
        waypoints = flip_h(waypoints, h_cond, v_cond, break_idx, idx_h, idx_v, d)
        break
    if (idx + 1) * wv <= L:
        waypoints.append([0, (idx+1)*wv])
    else: 
        v_cond, term = False, 3
        break_idx = idx + 1
        waypoints, d = flip_v(waypoints, h_cond, v_cond, break_idx, idx_h, idx_v)
        break
    if (idx + 2) * wv <= L:
        waypoints.append([0, (idx+2)*wv])
    else: 
        break_idx = idx + 2
        v_cond, term = False, 4
        waypoints, d = flip_v(waypoints, h_cond, v_cond, break_idx, idx_h, idx_v)
        break
    idx += 2
    

    
plot_waypoints(waypoints)

flip vertical
h_cond: True, v_cond: False


TypeError: flip_h() missing 1 required positional argument: 'd'